In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os, sys

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from pair_trading import generate_signals, backtest


In [5]:
ticker_a, ticker_b = "GDX", "GDXJ"  
start, end = "2015-01-01", None
lookback, entry_z, exit_z = 60, 2.0, 0.5
tc_bps, slippage_bps = 3.0, 1.5


In [3]:
def fetch_close(ticker, start, end):
    df = yf.download(ticker, start=start, end=end, progress=False)
    return df["Close"].rename(ticker).dropna()

a = fetch_close(ticker_a, start, end)
b = fetch_close(ticker_b, start, end)
data = pd.concat([a, b], axis=1).dropna()
data.tail()


YF.download() has changed argument auto_adjust default to True



1 Failed download:
['GDX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')

1 Failed download:
['GDXJ']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Ticker,GDX,GDXJ
Date,,


In [ ]:
df = generate_signals(a=data.iloc[:,0], b=data.iloc[:,1],
                      lookback=lookback, entry_z=entry_z, exit_z=exit_z)
df.tail()


In [ ]:
plt.figure()
df["z"].plot(title="Spread z-score")
plt.xlabel("Date"); plt.ylabel("z"); plt.show()


In [ ]:
equity, metrics = backtest(df, tc_bps=tc_bps, slippage_bps=slippage_bps)
print("Metrics:", metrics)

plt.figure()
equity.plot(title="Equity Curve")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.show()

os.makedirs("../reports", exist_ok=True)
equity.to_csv("../reports/equity_curve.csv")
df.to_csv("../reports/signals.csv")

import json
with open("../reports/metrics.json","w") as f:
    json.dump(metrics, f, indent=2)
